In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random

In [76]:
#creating functions for user input and recomendation

def user_input_song():
    song_input=input("input your favourite song:")
    print("")
    song_input=song_input.lower()
    k=0 #count of how many times we checked the charts
    return user_input_chart(song_input,k)

def user_input_chart(song,k):
    chart_input=input("Which chart do you want to check? Billboard or iTunes?")
    chart_input=chart_input.lower()
    song_input=song
    k=k
    if chart_input in ("itunes", "billboard"):
        if chart_input=="itunes":
            return webscrap(chart_input, song_input, k)
        else:
            return webscrap(chart_input, song_input, k)
    else:
        check=input("sorry, we don't have access to this chart yet. Do you want to try another one? yes/no:")
        if check.lower()=='yes':
            return user_input_chart(song_input)

def soup_func(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    return soup
        
def webscrap(chart, song, k):   
    song_input=song
    chart_input=chart
    k=k
    if chart_input=="itunes":
        #webscrapping iTunes
        url="https://www.popvortex.com/music/charts/top-100-songs.php"
        response=requests.get(url)
        soup_i=soup_func(url)
        #creating a lits of songs
        list_songs_itunes=[]
        for song in soup_i.select("cite.title"):
            list_songs_itunes.append(song.get_text(strip=True))

            #creating a lits of artists
        list_artists_itunes=[]
        for artist in soup_i.select("em.artist"):
            list_artists_itunes.append(artist.get_text(strip=True))

            #creating a dataframe

        df=pd.DataFrame({"song": list_songs_itunes, "artist":list_artists_itunes})
        df['source']='itunes'

        return recommendation(song_input, chart_input, df, k)
    else:
            #webscrapping billboard
        url="https://www.billboard.com/charts/hot-100/"
        response=requests.get(url)
        soup_b=soup_func(url)

            #creating a list of top-100 songs
        list_songs=[]
        for song in soup_b.select("div.chart-results-list h3#title-of-a-story.c-title.a-no-trucate.a-font-primary-bold-s.u-letter-spacing-0021"):
            list_songs.append(song.get_text(strip=True))

            #creating a list of top-100 artists for those songs
        list_art=[]
        for artist in soup_b.select("div.chart-results-list span.c-label.a-no-trucate.a-font-primary-s"):
            list_art.append(artist.get_text(strip=True))

            #creating a dataframe
        df=pd.DataFrame({"song": list_songs, "artist": list_art})
        df['source']='billboard'

        return recommendation(song_input, chart_input, df, k)

def recommendation(song, chart, df_to_check, k):
    i=0
    check=0
    k=k
    df=df_to_check
    for song_from_list in df_to_check['song']:
        if song!=song_from_list.lower():
            i=i+1
        else:
            check=1
            break
   
    if check==0 and k==0:
        other_chart_check=input(f"sorry your chosen song is not in the {chart.upper()} chart, do you want to try the other chart? Yes or no:")
        print("")
        if other_chart_check.lower()=="yes" and chart=="itunes":
            k+=1
            chart="billboard"
            return webscrap(chart, song,k)
        elif other_chart_check.lower()=="yes" and chart=="billboard":
            k+=1
            chart="itunes"
            return webscrap(chart, song,k)
        else:
            other_song_check=input(f"Do you want to check another song in {chart} chart maybe? yes/no:")
            if other_song_check.lower()=="yes":
                song_new=input(f"another song you want to check in {chart.upper()} chart:")
                song_new=song_new.lower()
                return recommendation(song_new, chart, df,k)
            else:
                print("whatever, see ya later!")
    elif check==0 and k!=0:
        if chart=="billboard":
            chart="itunes"
        else:
            chart="billboard"
            
        other_song_check=input(f"Sorry, not in the other list either. Do you want to check another song in {chart} chart maybe? yes/no:")
        if other_song_check.lower()=="yes":
            k=0
            song_new=input(f"another song you want to check in {chart.upper()} chart:")
            song_new=song_new.lower()
            return webscrap(chart, song_new, k)
        else:
            print("whatever, see ya later!")
    else:
        print(f"The song you've inputed is in {chart.upper()} chart!")
        print("")
        random_song = random.choice(df_to_check['song'])
        if random_song!=song.lower():
            row_w_song=df_to_check[df_to_check['song']==random_song]
            artist=row_w_song['artist'].iloc[0]
            print("You may like another one from the same chart, try this one:", random_song.upper(), "by", artist.upper())
            print("")
        else:
            random_song = random.choice(filtered_df['song'])
            row_w_song=df_to_check[df_to_check['song']==random_song]
            artist=row_w_song['artist'].iloc[0]
            print("You may like another one from the same chart, try this one:", random_song.upper(), "by", artist.upper())
            print("")
        choice=input('Do you want to try again? yes/no:')
        if choice.lower()=='yes':
            return user_input_song()
    

In [ ]:
user_input_song()

input your favourite song:bl

Which chart do you want to check? Billboard or iTunes?itunes
sorry your chosen song is not in the ITUNES chart, do you want to try the other chart? Yes or no:yes

Sorry, not in the other list either. Do you want to check another song in itunes chart maybe? yes/no:yes
another song you want to check in ITUNES chart:carnival
The song you've inputed is in ITUNES chart!

You may like another one from the same chart, try this one: BEAUTIFUL THINGS by BENSON BOONE

Do you want to try again? yes/no:yes
input your favourite song:made for me

Which chart do you want to check? Billboard or iTunes?billboard
The song you've inputed is in BILLBOARD chart!

You may like another one from the same chart, try this one: LA DIABLA by XAVI

Do you want to try again? yes/no:no
